In [1]:
!pip install scikit-learn


In [3]:
import cuml
import cupy as cp
import pandas as pd
from cuml.linear_model import LogisticRegression
from cuml.metrics import accuracy_score
import time

# Load and preprocess dataset
df = pd.read_csv("https://archive.ics.uci.edu/static/public/891/data.csv")
df = df.drop(columns=["ID"])
X = df.drop(columns=["Diabetes_binary"]).values
y = df["Diabetes_binary"].values

# Standard scaling using cuDF (optional but can use sklearn too)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to cupy arrays for RAPIDS
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# Train RAPIDS Logistic Regression
start = time.time()
clf = LogisticRegression()
clf.fit(X_train_gpu, y_train_gpu)
end = time.time()

# Predict and evaluate
y_pred_gpu = clf.predict(X_test_gpu)
acc = accuracy_score(y_test_gpu, y_pred_gpu)

print("RAPIDS cuML Logistic Regression")
print(f"Accuracy: {float(acc):.4f}")
print(f"Training time: {end - start:.4f} seconds")


RAPIDS cuML Logistic Regression
Accuracy: 0.8658
Training time: 0.0286 seconds


In [4]:
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.metrics import accuracy_score
import cupy as cp

# Convert data to GPU arrays
X_train_gpu = cp.asarray(X_train)
X_test_gpu = cp.asarray(X_test)
y_train_gpu = cp.asarray(y_train)
y_test_gpu = cp.asarray(y_test)

# RAPIDS Random Forest
start = time.time()
clf = cuRF(n_estimators=100, max_depth=10, random_state=42)
clf.fit(X_train_gpu, y_train_gpu)
end = time.time()

y_pred_gpu = clf.predict(X_test_gpu)
acc = accuracy_score(y_test_gpu, y_pred_gpu)

print("🚀 RAPIDS cuML Random Forest")
print(f"Accuracy: {float(acc):.4f}")
print(f"Training time: {end - start:.4f} seconds")


/home/ec2-user/anaconda3/envs/rapids/lib/python3.10/site-packages/cuml/internals/api_decorators.py:342: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/ec2-user/anaconda3/envs/rapids/lib/python3.10/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


🚀 RAPIDS cuML Random Forest
Accuracy: 0.8662
Training time: 2.3145 seconds
